# DIVE INTO ALGORITHMS
**Bradford Tuckfield**, No Starch Press, 2021

# Inledning

Vi kan betrakta en algoritm som en endlig mängd regler i en följd av operationer som löser en specifik typ av problem.

Datorer och andra digitala strukturer är bra verktyg till att implementera algoritmer med god precision, men algoritmer i allmänhet är mycket mera allmänna företeelser än specifika datorprogram. 

Dokumentet innehåller:

1. [Russian peasant multiplication](#russian_peasant_multiplication)

<a id='russian_peasant_multiplication'></a>
## Russian Peasant Multiplication (RPM)

Vi vill multiplicera 89 med 18 och vi bildar nu två kolumner, en för *halvering* och en för *dubblering*. 

I halveringskolumnen delar vi nu med 2 och ignorerar resten. Så fortsätter vi tills vi når 1. Och sen dubblerar vi iterativt i den andra kolumnen. Detta ger tabellen nedan.

<img src="images/algorithms01.png" style="width: 100px;"/>

Vi plockar nu bort alla värden i halveringskolumnen som är jämna tal. Och slutligen tar vi summan av det som är kvar i dubbleringskolumnen. Detta ger oss svaret på $89\times18=1602$.

<img src="images/algorithms02.png" style="width: 300px;"/>

Detta fungerar eftersom dubbleringskolonnen består av en summa av $2^a$ multipler av 18 och mera specifikt ser vi att vi har

$$
(2^0\cdot18) + (2^3\cdot18) + (2^4\cdot18) + (2^6\cdot18) = (1+2+4+8+16+32+64)\cdot 18 = 89\cdot18
$$

eftersom vi endast har kvar de $2^a$ som motsvarar jämna tal i halveringskolumnen. Men varför fungerar detta? 

Titta nu på hur tabellen ser ut om vi skriver halveringskolumnen som summor av $2^a$ termer och multiplicerar varje sådan summa med $2^1$. Vidare lägger vi till en term $2^0$ i varje rad där halveringstalet är udda.

<img src="images/algorithms03.png" style="width: 400px;"/>

Numrera nu raderna från 0 till *n* vilket i detta fall ger följden $\{0,1,2,3,4,5,6\}$. Intressant nog är det raderna 0, 3, 4 och 6 som är där vi har lagt till en extra $2^0$, alltså det är raderna där vi har udda halveringstal. Och samtidigt är dessa radnummer exakt samma värden som potenserna i $2^0+2^3+2^4+2^6$ gav oss ovan.

När vi således tar 

$$
18\cdot\left(2^0+2^3+2^4+2^6\right)
$$

är detta helt enkelt produkten av 18 med summan av de potenser av 2 som ger 89. Men detta måste då motsvara summan av exakt de raderna i dubbleringskolumnen som innehåller just dessa potenser eftersom vi i den kolumnen har brutit ned dubbleringen i termer av formen $18\cdot 2^a$.

Egentligen har vi i denna algoritm en annan algoritm inbakad i form av talet 89 på binär form eftersom 

$$
89 = (1\cdot 2^0) + (0\cdot2^1) + (0\cdot 2^2) + (1\cdot 2^3) + (1\cdot 2^4) + (0\cdot 2^5) + (1\cdot 2^6)
$$
och
$$
89_2 = 1001101
$$

och nu ser vi återigen indirekt, den här gången från positionerna i binära forman av 89, att rad 0, 3, 4 och 6 av dubbleringskolumnen är de som vi summerar ihop.

<a id='the_destination'></a>
### Python implementering

Låt oss implementera detta i Python. 

Det centrala verktyget vi behöver är $\texttt{math.floor}$ så vi kan halvera och runda ned utan att spara någon rest. 

Vi bildar en lista med den af faktorerna som ska halveras och sen använder vi en loop som först halvera och sen lägger till nya värdet i halveringslistan.

Vi har redan sett att algoritmen fungerar för hand med $89\cdot18$, så vi kan börja med att implementera den produkten i Python för att se att det faktiskt fungerar.

In [ ]:
# math biblioteket innehåller funktionen math.floor
import math

# startvärdet i rad 0
h0 = 89

# initiera halveringslistan och inkludera startvärdet 
halvering = [h0]

# loopa genom halveringar av sista värdet i listan tills sista värdet är 1
while halvering[-1] > 1:
    halvering.append(math.floor(halvering[-1]/2))

print(halvering)

Vi vill också bilda en lista för dubblering med startvärdet 18 och vi gör detta enkelt med en loop som utnyttjar antalet element i halveringslistan.

In [ ]:
# startvärdet i rad 0
d0 = 18

# initiera dubbleringslistan och inkludera startvärdet
dubblering = [d0]

# loopa genom dubbleringar av sista värdet i listan len(halvering)-1 antal gånger
# eftersom vi redan har första värdet = d0
for i in range(len(halvering)-1):
    dubblering.append(dubblering[-1]*2)

print(dubblering)

Vi vill nu göra en samlad tabell med våra två listor på samma sätt som vi gjorde för hand ovan. Med Pythons bibliotek $\texttt{pandas}$ kan man lätt göra detta. Vi använder en tvådimensionell dataform som kallas för $\texttt{DataFrame}$ som bland annat har fördelen att den direkt numrerar rader och kolumner från och med index 0. 

Vi bildar tabellen genom att använda funktionen $\texttt{zip}$ som klistrar ihop de två kolumnerna - som när en blixtlås samlar två sidor av en jacka.

Notera att när du kör koden nedan första gången kan det ta en stund innan cellens output visas. Pandaer är lite tröga djur.

In [ ]:
import pandas as pd

rpm_tabell = pd.DataFrame(zip(halvering, dubblering))

print(rpm_tabell)

Det återstår att både plocka bort raderna 1, 2 och 5 från kolumn 0 och sen att summera kvarvarande värden i kolumn 1.

I Pandas har vi en smidig kommando $\texttt{[namn].loc[ ]}$ som specifikt väljer ut de rader en en given kolumn som vi specificerar. Och i det här fallet vill vi komma åt alla rader som *inte* är delbara med 2 eftersom vi från kolumn 0 endast vill ha kvar rader med udda tal. 

För att göra detta kan vi använda Pythons inbyggda $\texttt{modulo}$ funktionalitet och välja alla talen som är 1 modulo 2, alltså alla tal som har en rest 1 när de delas med 2, vilket ju precis är alla udda tal. Alla sådana tal *a* skriver vi $\texttt{a%2 == 1}$.

$\texttt{loc}$ kommandot använder kommas till att separera rader och kolumner och eftersom det är från kolumn 0 vi vill identificera de udda talen är det också på dennas rader att vi gör modulooperationen. *Men* sen är det ju egentligen de kvarvarande värden i kolumn 1 vi vill ha kvar och vi låter alltså $\texttt{loc}$ välja att ha kvar just denna kolonn.

I Python blir detta

In [ ]:
rpm_values = rpm_tabell.loc[rpm_tabell[0]%2 == 1, 1]

Vi använder därmed tabellen till att bilda en ny tabell som bara är *en* kolumn med precis de värden vi vill summera.

Man kan nu direkt i Pandas summera alla värden i rpm_values tabellen men säg att vi gärna vill ha en array med värden först av någon anledning, till exempel om vi vill göra något annat med data eller vi vill spara dem i ett format vi lätt kan använda i loops och liknande. 

Vi använder därför Pandas $\texttt{.values}$ kommando som gör om DataFramen till en array. 

In [ ]:
rpm_values = rpm_values.values
print(rpm_values)

Slutligen summerar vi alla elementen i rpm_values och nu har vi resultatet av multiplikationen vi söker.

In [ ]:
rpm_svar = sum(rpm_values)
print(f'{h0} * {d0} = {rpm_svar}')

Vi kan nu samla hela koden nedan. I implementeringen vill vi gärna låta användaren kunna bestämma vilken produkt som ska beräknas så vi lägger till några linjer kod med inputmöjligheter samt inkluderar lite pedagogisk text.

In [ ]:
# samlat program för RPM multiplikationsalgoritmen
"""Detta program använder algoritmen Russian Peasant Multiplication (RPM) 
till att beräkna produkten av två tal."""

# math biblioteket innehåller funktionen math.floor
import math

#pandas biblioteket använder vi till att bilda en datatabell av två listor
import pandas as pd

# introducerande text
print('Detta program använder algoritmen Russian Peasant Multiplication (RPM)\n' 
      'till att beräkna produkten av två heltal.')

# halveringslistans startvärde i rad 0
h0 = int(input(f'\nSkriv in det första heltalet i produkten: '))

# initiera halveringslistan och inkludera startvärdet 
halvering = [h0]

# loopa genom halveringar av sista värdet i listan tills sista värdet är 1
while halvering[-1] > 1:
    halvering.append(math.floor(halvering[-1]/2))

# dubbleringsllistans startvärde i rad 0
d0 = int(input(f'\nSkriv in det andra heltalet i produkten: '))

# initiera dubbleringslistan och inkludera startvärdet
dubblering = [d0]

# loopa genom dubbleringar av sista värdet i listan len(halvering)-1 antal gånger
# eftersom vi redan har första värdet = d0
for i in range(len(halvering)-1):
    dubblering.append(dubblering[-1]*2)

print(f'\nBeräknar nu produkten {h0} * {d0}')

# bilda en tabell av de två listorna
rpm_tabell = pd.DataFrame(zip(halvering, dubblering))

# välj bara ut de värden i dubbleringslistan 
# som motsvaras av udda tal i halveringslistan.
rpm_values = rpm_tabell.loc[rpm_tabell[0]%2 == 1, 1]

# gör en array ut av värden vi har kvar från dubbleringslistan
rpm_values = rpm_values.values

# summera värden i arrayen och skriv ut svaret
rpm_svar = sum(rpm_values)
print(f'{h0} * {d0} = {rpm_svar}')